In [ ]:
# kneed is used to find knee of a prticular graph
# Here we are using it to find the optimum cluster size  s from the inertia v/s cluster graph
!pip install kneed

In [ ]:
# Importing all required libraries
import pandas as pd
import numpy as np
import sklearn

import matplotlib.pyplot as plt

In [ ]:
%rm -rf ML_project
!git clone https://github.com/ZubeenK/ML_project

Cloning into 'ML_project'...
remote: Enumerating objects: 19, done.
remote: Counting objects: 100% (19/19), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 19 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (19/19), done.


In [ ]:
# importing the data files 
trains_mdf = pd.read_csv("/content/ML_project/GQPD_work.csv")

In [ ]:
# creating a class to store required datas from the dataframe
class Train:
  id = -1
  strtStation = None
  endStation = None
  strtTime = None
  endTime = None
  journeyData=[]
  
  travelDays = None
  direction = None
  
  def __init__(self, id):
    self.id = id

In [ ]:
def checkTrain(train_id,trains_list):
  '''Fuction returns true if particular train id is not present in the list of class objects "trains_list"'''
  return np.all([train_id != data.id for data in trains_list])

In [ ]:
def directionChecker(drList):
  '''Returns the direction in which the train is moving
     P.S. Direction are predefined based on a defined algorithm'''
  if np.any([dr == 'down' for dr in drList]):
    return 'down'
  elif np.any([dr == 'up' for dr in drList]):
    return 'up'

In [ ]:
trains_mdf.head()

,Unnamed: 0,TRAIN,WEEKDAYS,SEQ,STATION,ARVL,STPG,DPRT,NEXT,BLCKSCTN,COABLCKSCTN,DAY,stationNameAgain,stationCode,Route,direction,mainBlocks,commonBlocks,totalBlocks
0,581,11015,"1,1,1,1,1,1,1",95,ET,46800,600,47400,48000,ET-PRKD,ET-PRKD,1,ET,4083,2_NDLS-MAS,down,1.0,0.0,1.0
1,582,11015,"1,1,1,1,1,1,1",96,PRKD,48000,0,48000,48480,PRKD-HBD,PRKD-HBD,1,PRKD,4082,2_NDLS-MAS,down,1.0,0.0,1.0
2,583,11015,"1,1,1,1,1,1,1",97,HBD,48480,120,48600,49080,HBD-BNI,HBD-BNI,1,HBD,4081,2_NDLS-MAS,down,1.0,0.0,1.0
3,584,11015,"1,1,1,1,1,1,1",98,BNI,49080,0,49080,49860,BNI-MIG,BNI-MIG,1,BNI,4080,2_NDLS-MAS,down,1.0,0.0,1.0
4,585,11015,"1,1,1,1,1,1,1",99,MIG,49860,0,49860,50220,MIG-CHQ,MIG-CHQ,1,MIG,4079,2_NDLS-MAS,down,1.0,0.0,1.0


In [ ]:
trainsList = []
#checks if for each train id present in the dataframe, whether it is present in the list of objects "Train" and
#adds it to the list if abscent 
for trainNo in trains_mdf['TRAIN']:
  if checkTrain(trainNo,trainsList):
    newTrain = Train(trainNo)
    currJourney = trains_mdf.loc[(trains_mdf['TRAIN'] == trainNo)]
    newTrain.journeyData = list(currJourney['STATION'])[:]
    newTrain.strtStation = newTrain.journeyData[0]
    newTrain.endStation = newTrain.journeyData[-1]
    temp = list(currJourney['WEEKDAYS'])
    newTrain.travelDays = temp[0]
    temp = list(currJourney['direction'])
    newTrain.direction = directionChecker(temp)
    temp = list(currJourney['ARVL'])
    newTrain.strtTime = temp[0]
    newTrain.endTime = temp[-1]
    trainsList.append(newTrain)

In [ ]:
print(trainsList[0].id)
print(trainsList[0].strtStation)
print(trainsList[0].endStation)
print(trainsList[0].journeyData)
print(trainsList[0].strtTime)
print(trainsList[0].endTime)
print(trainsList[0].travelDays)
print(trainsList[0].direction)

11015
ET
JHS
['ET', 'PRKD', 'HBD', 'BNI', 'MIG', 'CHQ', 'BKA', 'ODG', 'ITKL', 'MDDP', 'MSO', 'HBJ', 'BPL', 'NSZ', 'SUW', 'BVB', 'DWG', 'SMT', 'SCI', 'BHS', 'SORI', 'SUMR', 'GLG', 'PAI', 'BAQ', 'BET', 'CLHT', 'KAH', 'MABA', 'BINA', 'AGD', 'KOA', 'MXS', 'DUA', 'JLN', 'JRO', 'LAR', 'DWA', 'JHA', 'BJA', 'DLA', 'TBT', 'MZX', 'BZY', 'BPW', 'BAB', 'KHJ', 'BJI', 'JHSA', 'JHS']
46800
69420
1,1,1,1,1,1,1
down


In [ ]:
#Creating a list of Non Daily Trains
trainsNonDaily = []
for train in trainsList:
  if len(train.journeyData)>1:
    if train.travelDays != '1,1,1,1,1,1,1':
      trainsNonDaily.append(train)

In [ ]:
#Creating a split on the Non Daily Trains based on "Up" or "Down" direction
trainsNonDailyUp = []
trainsNonDailyDown = []

for train in trainsNonDaily:
  if train.direction == 'up':
    trainsNonDailyUp.append(train)
  elif train.direction == 'down':
    trainsNonDailyDown.append(train)
  else:
    print("Train # {} has direction {}".format(train.id,train.direction))


Train # 11125 has direction None
Train # 12547 has direction None
Train # 12548 has direction None
Train # 15563 has direction None
Train # 15564 has direction None
Train # 19057 has direction None
Train # 19058 has direction None
Train # 19063 has direction None
Train # 19064 has direction None


In [ ]:
# Importing station list for further use
stationDF = pd.read_csv("/content/ML_project/stationData_route2.csv")
stationDF

,Name,startKM,endKM,stationEntrySpeed(kmph)
0,NDLS,0.00,0.75,15.0
1,CSB,1.25,2.00,15.0
2,TKJ,2.62,3.37,15.0
3,PGMD,4.23,4.98,15.0
4,NZM,7.25,8.00,15.0
...,...,...,...,...
296,VOC,2173.94,2174.69,15.0
297,TNP,2176.44,2177.19,15.0
298,KOK,2177.59,2178.34,15.0
299,BBQA,2179.59,2180.34,15.0


In [ ]:
# Creating a list of stations based on required
stationList = list(stationDF['Name'])

In [ ]:
def routeChecker(train):
  '''Fuction defined to check the route of given train data'''
  travelStation = train.journeyData
  CheckInfo = []
  for i in range(len(travelStation)):
    if travelStation[i] in stationList:
      CheckInfo.append(1)
    else:
      CheckInfo.append(0)
  return CheckInfo

In [ ]:
# creating a list of IDs of Non Daily trains in Down Direction and as well as the list of travel days of the trains
trainIDList = [train.id for train in trainsNonDailyDown]
trainDateList = [train.travelDays for train in trainsNonDailyDown]
startTimeList = [train.strtTime for train in trainsNonDailyDown]
endTimeList = [train.endTime for train in trainsNonDailyDown]


In [ ]:
# Creating a new refined DF that is to be used for clustering
FinalTrainDF = pd.DataFrame()
FinalTrainDF['Train ID'] = trainIDList
FinalTrainDF['Weekdays'] = trainDateList
FinalTrainDF['STRT TIME'] = startTimeList

FinalTrainDF['END TIME'] = endTimeList
for station in stationList:
  FinalTrainDF[station] = None
# Distinguishes the list of stations a particular train visits
for i in FinalTrainDF.index:
  for station in stationList:
    if station in trainsNonDailyDown[i].journeyData:
      FinalTrainDF[station][i] = 1
    else:
      FinalTrainDF[station][i] = 0
display(FinalTrainDF)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Train ID,Weekdays,STRT TIME,END TIME,NDLS,CSB,TKJ,PGMD,NZM,OKA,TKD,TKDC,FDB,FDN,BVH,AST,PWL,RDE,SHLK,BNCR,HDL,KSV,CHJ,AJH,VRBD,BTSR,MTJ,BAD,FAR,DDDM,KXM,RNKA,BFP,RKM,AGC,BHA,JJ,MIA,DHO,HET,...,SVPM,BTTR,AXR,TMC,KJJ,PGU,NLR,NLS,VDE,VKT,KMLP,MBL,GDR,ODUR,PYA,NYP,DVR,PEL,SPE,AKAT,TADA,AKM,ELR,GPD,KVP,PON,APB,MJR,NPKM,AIP,AIPP,ENR,KAVM,WCN,TVT,VOC,TNP,KOK,BBQA,MASA
0,11045,"0,0,0,0,0,1,0",102240,130200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,11050,"0,0,0,0,0,1,0",85500,106800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,11079,"0,0,0,0,1,0,0",99600,120330,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,11088,"0,0,0,1,0,0,0",85500,106800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,11090,"0,0,0,0,0,0,1",85500,107055,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,22997,"0,0,1,1,0,0,1",57480,68100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
336,82402,"0,0,1,0,1,0,0",53340,65100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
337,82404,"1,0,0,0,0,0,0",53340,65100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
338,82653,"0,0,0,0,1,0,0",130200,161100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Creating a data set for checking
X_train = FinalTrainDF.drop(['Train ID','Weekdays'],axis = 1)
X_train

,NDLS,CSB,TKJ,PGMD,NZM,OKA,TKD,TKDC,FDB,FDN,BVH,AST,PWL,RDE,SHLK,BNCR,HDL,KSV,CHJ,AJH,VRBD,BTSR,MTJ,BAD,FAR,DDDM,KXM,RNKA,BFP,RKM,AGC,BHA,JJ,MIA,DHO,HET,SIKD,MRA,SANK,NUB,...,BTTR,AXR,TMC,KJJ,PGU,NLR,NLS,VDE,VKT,KMLP,MBL,GDR,ODUR,PYA,NYP,DVR,PEL,SPE,AKAT,TADA,AKM,ELR,GPD,KVP,PON,APB,MJR,NPKM,AIP,AIPP,ENR,KAVM,WCN,TVT,VOC,TNP,KOK,BBQA,MASA,Cluster No
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5
336,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24
337,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,24
338,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10


In [ ]:
# Kmeans Clustering algorithms
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
sil = []
kmax = 100

# dissimilarity would not be defined for a single cluster, thus, minimum number of clusters should be 2
#for k in range(2, 50):
  #kmeans = KMeans(n_clusters = k).fit(X_train)
  #labels = kmeans.labels_
  #sil.append(silhouette_score(X_train, labels, metric = 'euclidean'))

In [ ]:
#plt.figure(figsize=(20,15))
#plt.plot(range(2, 50), sil)
#plt.xticks(range(2, 50))
#plt.xlabel("Number of Clusters")
#plt.ylabel("SIL")
#plt.show()

In [ ]:
#kmeans = KMeans(n_clusters = k).fit(X_train)

In [ ]:
#kmeans_kwargs = {"init": "random", "n_init": 10, "max_iter": 300,"random_state": 42}
#sse = []
#for k in range(1, 20):
  #kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
  #kmeans.fit(X_train)
  #sse.append(kmeans.inertia_)

In [ ]:
#plt.plot(range(1, 20), sse)
#plt.xticks(range(1, 20))
#plt.xlabel("Number of Clusters")
#plt.ylabel("SSE")
#plt.show()

In [ ]:
#from kneed import KneeLocator


In [ ]:
#kl = KneeLocator(range(1, 20), sse, curve="convex", direction="decreasing")
#kl.elbow

In [ ]:
kmeans = KMeans(n_clusters = 30).fit(X_train)
labels = kmeans.labels_
FinalTrainDF['Cluster No'] = labels

In [ ]:
FinalTrainDF

,Train ID,Weekdays,STRT TIME,END TIME,NDLS,CSB,TKJ,PGMD,NZM,OKA,TKD,TKDC,FDB,FDN,BVH,AST,PWL,RDE,SHLK,BNCR,HDL,KSV,CHJ,AJH,VRBD,BTSR,MTJ,BAD,FAR,DDDM,KXM,RNKA,BFP,RKM,AGC,BHA,JJ,MIA,DHO,HET,...,BTTR,AXR,TMC,KJJ,PGU,NLR,NLS,VDE,VKT,KMLP,MBL,GDR,ODUR,PYA,NYP,DVR,PEL,SPE,AKAT,TADA,AKM,ELR,GPD,KVP,PON,APB,MJR,NPKM,AIP,AIPP,ENR,KAVM,WCN,TVT,VOC,TNP,KOK,BBQA,MASA,Cluster No
0,11045,"0,0,0,0,0,1,0",102240,130200,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,8
1,11050,"0,0,0,0,0,1,0",85500,106800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26
2,11079,"0,0,0,0,1,0,0",99600,120330,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10
3,11088,"0,0,0,1,0,0,0",85500,106800,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26
4,11090,"0,0,0,0,0,0,1",85500,107055,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,22997,"0,0,1,1,0,0,1",57480,68100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
336,82402,"0,0,1,0,1,0,0",53340,65100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
337,82404,"1,0,0,0,0,0,0",53340,65100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
338,82653,"0,0,0,0,1,0,0",130200,161100,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,18


In [ ]:
#importing the results
FinalTrainDF.to_csv('updated_Train_cluster(20 clusters).csv')